In [21]:
import numpy as np
import pandas as pd
import string

def rand(a,b):
    return (b-a)*np.random.random_sample() + a

def sigmoid(x):
    return np.tanh(x)
    # return 1./(1.+np.exp(-x)) # range 0 to 1

vsigmoid = np.vectorize(sigmoid)

def dsigmoid(y):
    return 1.0 - y**2

vdsigmoid = np.vectorize(dsigmoid)

class BPNN:
    def __init__(self, input_n, hidden_n, output_n):
        # 输入层、隐藏层、输出层的节点（数）
        self.input_n = input_n + 1 
        self.hidden_n = hidden_n
        self.output_n = output_n

        # 激活神经网络的所有节点（向量）
        self.input_cells = np.ones((self.input_n,1), dtype=float)
        self.hidden_cells = np.ones((self.hidden_n,1), dtype=float)
        self.output_cells = np.ones((self.output_n,1), dtype=float)

        # 建立权重（矩阵）
        # 输入层权重在 [-0.2, 0.2)
        self.input_weights = (np.random.random_sample((self.input_n, self.hidden_n)) - 0.5) * 0.4
        # 输出层权重在 [-2., 2.)
        self.output_weights = (np.random.random_sample((self.hidden_n, self.output_n)) - 0.5) * 4.
        
        # 最后一次权重改变的记录
        self.input_correction = np.zeros((self.input_n, self.hidden_n))
        self.output_correction = np.zeros((self.hidden_n, self.output_n))

    def update(self, inputs):
        if len(inputs) != self.input_n - 1:
            raise ValueError("Wrong number of inputs")

        # 激活输入层
        for i in range(self.input_n-1):
            self.input_cells[i,0] = inputs[i]

        # 激活隐藏层
        # shapes: (hidden_n,1) = (hidden_n,sinput_n) x (sinput_n,1)
        self.hidden_cells = vsigmoid( self.input_weights.T.dot(self.input_cells) )

        # 激活输出层
        # shapes: (output_n,1) = (output_n,hidden_n) x (hidden_n,1)
        self.output_cells = vsigmoid( self.output_weights.T.dot(self.hidden_cells) )

        return self.output_cells

    def backPropagate(self, targets, learning_rate, momentum_factor):
        ''' 反向传播 '''
        if len(targets) != self.output_n:
            raise ValueError('Wrong number of target values')

        # 计算输出层的误差 (output_n,1)
        error = targets - self.output_cells
        # (output_n,1) * error:
        output_deltas = vdsigmoid(self.output_cells) * error

        # 计算隐藏层的误差
        error = self.output_weights.dot(output_deltas)
        # (hidden_n,1) * error:
        hidden_deltas = vdsigmoid(self.hidden_cells) * error

        # 更新输出层权重
        change = self.hidden_cells.dot(output_deltas.T)
        self.output_weights += learning_rate*change + momentum_factor*self.output_correction
        self.output_correction = change

        # 更新输入层权重
        change = self.input_cells.dot(hidden_deltas.T)
        self.input_weights += learning_rate*change + momentum_factor*self.input_correction
        self.input_correction = change

        # 计算误差
        error = np.sum(0.5*(targets - self.output_cells)**2)
        return error

    def test(self, patterns):
        #y = []
        true = 0
        for p in patterns:
            a = p[1]
            b = self.update(p[0])
            print(p[1], '->', b)
            y_pred = []
            for n in b:
                if n==max(b):
                    y_pred.append(1)
                else:
                    y_pred.append(0)
            if a.index(1)==y_pred.index(1):
                true+=1
        print('accuracy:', true/len(patterns)*100, '%')
            
    def predict(self, patterns):
        y = []
        for p in patterns:
            b = self.update(p)
            y_pred = []
            for n in b:
                if n==max(b):
                    y_pred.append(1)
                else:
                    y_pred.append(0)
            y.append(y_pred)
        return y
    
    def weights(self):
        print('Input weights:')
        for i in range(self.input_n):
            print(self.input_weights[i])

        print()
        print('Output weights:')
        for j in range(self.hidden_n):
            print(self.output_weights[j])

    def train(self, patterns, iterations=1000, learning_rate=1, momentum_factor=0.1):
        # learning_rate: 学习率
        # momentum_factor: 动量因子
        errs = []
        for i in range(iterations):
            error = 0.0
            for p in patterns:
                inputs = p[0]
                if type(p[1]) == list:
                    targets = p[1]
                else: targets = list([p[1]])
                self.update(inputs)
                error = error + self.backPropagate(np.array(targets).reshape((len(targets),1)), learning_rate, momentum_factor) 
            errs.append(error)
            if i % 100 == 0:
                print('error %-0.5f' % error)

In [22]:
pat = [
        [[0,0], [0,1]],
        [[0,1], [1,0]],
        [[1,0], [1,0]],
        [[1,1], [0,1]]
    ]

In [26]:
nn = BPNN(2,5,2)
nn.train(pat)
nn.test(pat)

error 2.10085
error 1.51797
error 0.62868
error 0.51877
error 0.32017
error 0.29426
error 0.45111
error 0.22669
error 0.37975
error 0.18346
[0, 1] -> [[0.40042398]
 [0.9880083 ]]
[1, 0] -> [[0.96167765]
 [0.09262052]]
[1, 0] -> [[0.95832677]
 [0.32582805]]
[0, 1] -> [[0.49223469]
 [0.9922584 ]]
accuracy: 100.0 %


In [27]:
nn.predict([[0,0],[1,0],[0,1],[1,1]])

[[0, 1], [1, 0], [1, 0], [0, 1]]